In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.models as models
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
from PIL import Image
import pandas as pd
from tqdm.notebook import tqdm
import seaborn as sns

In [8]:
class KnifeDataset(Dataset):
    def __init__(self, knife_dir, non_knife_dir, transforms=None):
        self.transforms = transforms
        self.knife_imgs = list(sorted(os.listdir(knife_dir)))
        self.non_knife_imgs = list(sorted(os.listdir(non_knife_dir)))
        self.knife_dir = knife_dir
        self.non_knife_dir = non_knife_dir

    def __getitem__(self, idx):
        if idx < len(self.knife_imgs):
            img_path = os.path.join(self.knife_dir, self.knife_imgs[idx])
            label = 1  # Knife
        else:
            img_path = os.path.join(self.non_knife_dir, self.non_knife_imgs[idx - len(self.knife_imgs)])
            label = 0  # Non-Knife

        img = Image.open(img_path).convert("RGB")

        if self.transforms:
            img = self.transforms(img)

        return img, label

    def __len__(self):
        return len(self.knife_imgs) + len(self.non_knife_imgs)


In [19]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Initialize the dataset
knife_dir = r'D:\Weapon Detection Project DRDO\KnivesImagesDatabase\POSITIVES_ALL'
non_knife_dir = r'D:\Weapon Detection Project DRDO\KnivesImagesDatabase\NEGATIVES_ALL'
dataset = KnifeDataset(knife_dir, non_knife_dir, transforms=transform)

# Split the dataset into train and validation sets
indices = torch.randperm(len(dataset)).tolist()
split = int(0.8 * len(indices))  # 80-20 train-validation split
dataset_train = torch.utils.data.Subset(dataset, indices[:split])
dataset_valid = torch.utils.data.Subset(dataset, indices[split:])


In [20]:
data_loader_train = DataLoader(
    dataset_train, batch_size=32, shuffle=True, num_workers=4)

data_loader_valid = DataLoader(
    dataset_valid, batch_size=32, shuffle=False, num_workers=4)

In [21]:
# Load a pre-trained model
model = models.resnet50(pretrained=True)

# Replace the last layer
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 2)  # Binary classification (knife, non-knife)

# Move the model to the GPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [22]:
!jupyter nbextension enable --py widgetsnbextension --sys-prefix
!jupyter nbextension install --py widgetsnbextension --sys-prefix

usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: dejavu events execute kernel kernelspec lab
labextension labhub migrate nbconvert notebook run server troubleshoot trust

Jupyter command `jupyter-nbextension` not found.
usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [-

In [23]:
# Ensure the custom dataset and DataLoader definitions are in place
import torch
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os

class KnifeDataset(Dataset):
    def __init__(self, knife_dir, non_knife_dir, transforms=None):
        self.transforms = transforms
        self.knife_imgs = list(sorted(os.listdir(knife_dir)))
        self.non_knife_imgs = list(sorted(os.listdir(non_knife_dir)))
        self.knife_dir = knife_dir
        self.non_knife_dir = non_knife_dir

    def __getitem__(self, idx):
        if idx < len(self.knife_imgs):
            img_path = os.path.join(self.knife_dir, self.knife_imgs[idx])
            label = 1  # Knife
        else:
            img_path = os.path.join(self.non_knife_dir, self.non_knife_imgs[idx - len(self.knife_imgs)])
            label = 0  # Non-Knife

        img = Image.open(img_path).convert("RGB")

        if self.transforms:
            img = self.transforms(img)

        return img, label

    def __len__(self):
        return len(self.knife_imgs) + len(self.non_knife_imgs)

# Transformations
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Initialize Dataset and DataLoader
knife_dir = r'D:\Weapon Detection Project DRDO\KnivesImagesDatabase\NEGATIVES_ALL'
non_knife_dir = r'D:\Weapon Detection Project DRDO\KnivesImagesDatabase\NEGATIVES_ALL'
dataset = KnifeDataset(knife_dir, non_knife_dir, transforms=transform)

# Split Dataset
indices = torch.randperm(len(dataset)).tolist()
split = int(0.8 * len(indices))  # 80-20 train-validation split
dataset_train = torch.utils.data.Subset(dataset, indices[:split])
dataset_valid = torch.utils.data.Subset(dataset, indices[split:])

data_loader_train = DataLoader(dataset_train, batch_size=32, shuffle=True, num_workers=4)
data_loader_valid = DataLoader(dataset_valid, batch_size=32, shuffle=False, num_workers=4)

# Check DataLoader
print("Checking DataLoader...")

for idx, (images, labels) in enumerate(data_loader_train):
    if idx == 0:
        print(f"First batch loaded successfully, batch size: {len(images)}")
    if idx == 1:
        print("Second batch loaded successfully")
        break

print("DataLoader check completed.")


Checking DataLoader...
